In [1]:
import pandas as pd
from geopy import distance as dis
import itertools
import numpy as np
import re
import fuzzywuzzy
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

D:\Applications\Anaconda\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
def calculate_distance(p1,p2):
    return dis.distance(p1,p2).km

In [4]:
sm_gt=pd.read_csv("../data/sm_gt.csv")
all_infor_gt=pd.read_csv('../data/page_info_Guatemala.csv',delimiter='|',index_col=0)
infor_gt=pd.read_csv('../data/pages_Guatemala_validate.csv',delimiter='|',index_col=0)
matched_df_gt=pd.read_csv('../data/address_name_match_gt.csv')

In [5]:
infor_gt=infor_gt[['fb_id','latitude','longitude']].merge(all_infor_gt,left_on='fb_id',how='left',right_on="id")

In [6]:
name_list_gt=list(map(lambda x: x[1]['username'] if type(x[1]['username'])==str else x[1]['name'],infor_gt.iterrows()))

In [7]:
name_list_gt_space=list(map(lambda x :re.sub("([a-z])([A-Z])","\g<1> \g<2>",x),name_list_gt))

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [9]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))

In [10]:
vectorizer.fit(name_list_gt_space)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [11]:
def top1(string):
    df=pd.DataFrame([vectorizer.get_feature_names(),vectorizer.transform([string]).todense()[0].tolist()[0]]).T.sort_values(1,ascending=False)
    df=df[df[1]==df[1].max()]
    df.columns=['topic','score']
    df['string']=string
    tmp_list=df.reset_index(drop=True).values.tolist()
    return [' '.join(df['topic']),df['score'].max(),set(df['string']).pop()]
top1('Cruz Roja GTSanto Tomas Castilla')

['castilla tomas gtsanto',
 0.5613704932281236,
 'Cruz Roja GTSanto Tomas Castilla']

In [12]:
topic_sm_df_gt=pd.DataFrame(list(map(top1,name_list_gt_space)))

In [13]:
topic_sm_df_gt.columns=['topic','score','name_sm']

In [14]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
vectorizer.fit(matched_df_gt['name'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [15]:
topic_match_df_gt=pd.DataFrame(list(map(top1,matched_df_gt['name'])))

In [16]:
topic_match_df_gt.columns=['topic','score','name_match']

In [17]:
def fuzzy_match(string):
    match=process.extract(string,topic_match_df_gt['topic'],limit=1,scorer=fuzz.token_sort_ratio)[0]
    return [string,match[0]]

In [18]:
middle_match_df_gt=pd.DataFrame(list(map(fuzzy_match,topic_sm_df_gt['topic'])))

In [19]:
middle_match_df_gt.columns=['sm_topic','name_topic']

In [20]:
tmp_df_gt=topic_sm_df_gt.merge(middle_match_df_gt,left_on='topic',how='left',right_on='sm_topic')[["name_sm","sm_topic","name_topic"]].merge(topic_match_df_gt,how='left',left_on='name_topic',right_on='topic')[['name_sm','name_match']].merge(matched_df_gt,how='left',left_on="name_match",right_on='name').drop(columns=['Unnamed: 0','min_dist','name'])

In [21]:
merged_gt=pd.concat([infor_gt,tmp_df_gt],axis=1)[['fb_id', 'latitude', 'longitude','name', 'phone', 'username', 'website', 'single_line_address','name_sm',
       'name_match', 'address', 'web_lat', 'web_lng', 'weight']]

In [22]:
merged_gt['distance_page_web']=merged_gt.apply(lambda x:calculate_distance((x['latitude'],x['longitude']),(x['web_lat'],x['web_lng'])),axis=1)

In [23]:
Gmaps_gt_df=pd.read_csv('../data/GMaps_Guatemala.csv',index_col=0)

In [24]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
vectorizer.fit(Gmaps_gt_df['place_name'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [25]:
topic_Gmap_df_gt=pd.DataFrame(list(map(top1,Gmaps_gt_df['place_name'])))

In [26]:
topic_Gmap_df_gt.columns=['topic','score','name_match']

In [27]:
def fuzzy_match(string):
    match=process.extract(string,topic_Gmap_df_gt['topic'],limit=1,scorer=fuzz.token_sort_ratio)[0]
    return [string,match[0]]

In [28]:
middle_match_df_gt=pd.DataFrame(list(map(fuzzy_match,topic_sm_df_gt['topic'])))

In [29]:
middle_match_df_gt.columns=['sm_topic','Gmaps_topic']

In [30]:
tmp_df_gt=topic_sm_df_gt.merge(middle_match_df_gt,left_on='topic',how='left',right_on='sm_topic')[["name_sm","sm_topic","Gmaps_topic"]].merge(topic_Gmap_df_gt,how='left',left_on='Gmaps_topic',right_on='topic')[['name_sm','name_match']].merge(Gmaps_gt_df,how='left',left_on="name_match",right_on='place_name').drop(columns=['place_id','name_match'])[['name_sm','place_name','place_address','place_lat','place_long']].drop_duplicates().reset_index(drop=True)

In [31]:
tmp_df_gt.columns=['name_sm', 'Gmaps_place_name', 'Gmaps_place_address', 'Gmap_lat', 'Gmap_long']

In [32]:
merged_gt=merged_gt.merge(tmp_df_gt,left_on='name_sm',how='left',right_on='name_sm')

In [33]:
merged_gt['distance_page_Gmaps']=merged_gt.apply(lambda x:calculate_distance((x['latitude'],x['longitude']),(x['Gmap_lat'],x['Gmap_long'])),axis=1)

In [34]:
merged_gt.columns=['fb_id', 'fb_latitude', 'fb_longitude', 'name', 'phone', 'username',
       'website', 'single_line_address', 'name_sm', 'name_match', 'web_address',
       'web_lat', 'web_lng', 'weight', 'distance_page_web', 'Gmaps_place_name',
       'Gmaps_place_address', 'Gmap_lat', 'Gmap_long', 'distance_page_Gmaps']

In [35]:
merged_gt.to_csv("../data/validation_set_gt.csv")